In [ ]:
# generate prompts
from openai import OpenAI
import json
import os
import random
import tqdm

api_key = ""
client = OpenAI(api_key=api_key)
data_dir = "../../data/ai_detection_3000.raw_data.json"
dataset=data_dir.split("/")[-1].split(".")[0]
model="gpt-3.5-turbo"

with open(data_dir,"r") as f:
    sentences = json.load(f)['original']

result = dict(original=sentences,rewritten=[]) 
prompts = dict(inp_prompt=[], out_prompt=[])    

if "pubmed" in data_dir:
    questions = [i.split("Answer:")[0] for i in sentences] 
    input_sentences = [i.split("Answer: ")[1] for i in sentences]
    print(questions[0])
else:
    input_sentences=sentences

print(len(input_sentences))
print(input_sentences[0])

word_lens = [15,30,50] 
styles=["formal", "oral", "academic", "literary", "critical", "narrative", "descriptive", "lyric", "objective", "subjective"] 

for _ in tqdm.tqdm(range(input_sentences)):
    
    word_len=random.choice(word_lens)
    style=random.choice(styles)
    prompt1 = f"Write a prompt in {word_len} words that says you want gpt's help in polishing a paragraph in a {style} style, this prompt can only be {word_len} words or less."
    prompts['inp_prompt'].append(prompt1)
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": f"{prompt1}"}
        ],
        temperature=0.7
    )
    prompt2 = completion.choices[0].message.content
    prompts['out_prompt'].append(prompt2)
    
with open(f'../../data/ai_detection_polish_3000_prompt.json', 'w', encoding='utf-8') as f: # 保存prompt
    json.dump(prompts, f, ensure_ascii=False, indent=4)

In [ ]:
from openai import OpenAI
import json
import os
import random
import tqdm

api_key = ""
client = OpenAI(api_key=api_key)

data_dir = "../../data/portuguese_train_500.json" 
dataset="portuguese_train_500"
model="gpt-3.5-turbo" 
prompt_json_path = "../../data/ai_detection_500_polish_prompt.json"
resume = False

with open(data_dir,"r") as f:
    sentences = json.load(f)['original']

if resume:
    with open(f"../../data/polish/{model}/{dataset}_polish_{model}.raw_data.json",'r') as f:
        result = json.load(f)
else:
    result=dict(original=sentences,rewritten=[])
    
print(len(result['rewritten']))
print(len(result['original']))

if "pubmed" in data_dir: 
    questions = [i.split("Answer:")[0] for i in sentences]
    input_sentences = [i.split("Answer: ")[1] for i in sentences]
    print(questions[0])
else:
    input_sentences=sentences
    
with open(prompt_json_path,"r") as f:
    prompts = json.load(f)['out_prompt']

print(len(input_sentences))
print(input_sentences[0])

for idx, sentences in enumerate(tqdm.tqdm(input_sentences[len(result['rewritten']):]),start=len(result['rewritten'])):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": f"{prompts[idx]} The output language should be Portuguese: \n{sentences}"}
        ],
        temperature=0.7
    )
    if "pubmed" in data_dir:
        result['rewritten'].append(questions[idx] + "Answer: " + completion.choices[0].message.content)
    else:
        result['rewritten'].append(completion.choices[0].message.content)
        print(completion.choices[0].message.content)

with open(f'../../data/polish/{model}/{dataset}_polish_{model}.raw_data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [ ]:
# if the above block get stuck, run this to save and set resume to True
with open(f'polish/{model}/{dataset}_polish_{model}.raw_data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)